## **[LeetCode Link](https://leetcode-cn.com/problems/k-th-smallest-prime-fraction/solution/di-k-ge-zui-xiao-de-su-shu-fen-shu-by-leetcode/)**

## 堆
### 思路
使用一个堆记录所有以 `primes[j]` 为分母且未被弹出的最小分数。依次从堆中弹出 `K-1` 个元素，此时堆顶的分数就是结果。
### 算法
在 `Python` 中，使用 `(fraction, i, j)` 表示分数 `fraction = primes[i] / primes[j]` 。如果下一个分数有效（即 `i+1 < j` ），那么使用当前分数时，将下一个分数压入堆中。
### 复杂度分析
时间复杂度：$O(K \log N)$，其中 `N` 是 `A` 的长度，堆中最多压入 `N` 个元素，每次弹出为 $O(\log N)$，需要 $O(K)$ 次弹出操作。
空间复杂度：$O(N)$，堆的大小。

In [ ]:
def kthSmallestPrimeFraction(self, A: List[int], K: int) -> List[int]:
        heap = [(A[0] / float(A[i]),0,i) for i in range(len(A)-1, -1, -1)]
        for t in range(K-1):
            now, i, j = heapq.heappop(heap)
            if i < j + 1:
                heapq.heappush(heap, (A[i+1] / A[j],i+1,j))
        now, i, j = heapq.heappop(heap)
        return [A[i], A[j]]

In [ ]:
class Solution {
    public int[] kthSmallestPrimeFraction(int[] A, int K) {
        PriorityQueue<int[]> pq = new PriorityQueue<int[]>((a, b) ->
                A[a[0]] * A[b[1]] - A[a[1]] * A[b[0]]);

        for (int i = 1; i < A.length; ++i)
            pq.add(new int[]{0, i});

        while (--K > 0) {
            int[] frac = pq.poll();
            if (frac[0]++ < frac[1] + 1)
                pq.offer(frac);
        }

        int[] ans = pq.poll();
        return new int[]{A[ans[0]], A[ans[1]]};
    }
}

## 二分查找 + DP（复杂度压缩）
二分查找左值为最小分数，右值为最大分数，计算中值，从数组中找到所组成的分数比中值小的数对个数，并保留这些数对中最大的一组数对。
  * 如果比中值小的数对个数等于K，那么保留的最大数对就是答案，因为他是所有数对中前K小个数对中最大的一个，也就是第K小的数对；
  * 如果比中值小的数对个数小于K，说明中值取小了，令左值为中值；
  * 如果比中值小的数对个数大于K，说明中值取大了，令右值为中值。

使用DP计算数组中比中值小的数对个数 `count`，计算方法是从 `i` 之后找到第一个 `j` 使得 `A[i]/A[j] < mid`，此时从 `A[j]` 到 `A[n-1]` 都满足要求， `count += n - j` ，同时 `A[j]` 是能和 `A[i]` 组成最大分数的数，保留该最大数对。还有个关键的地方是 `j` 不用每次都从 `i+1` 开始循环，因为如果上一次对于 `A[i]` 第一个符合条件的是 `A[j]` ，那么对于 `A[i+1] > A[i]` ，想要让 `A[i+1]/A[x] < mid` ，那么 `x` 一定大于 `j` ，所以第一个满足 `A[i+1]` 的 `j` 一定比满足 `A[i]` 的 `j` 要大，所以 `j` 每次从上一次结束的位置继续遍历就可以。这样时间复杂度从 $\mathcal{O}(n^2)$ 减到了 $\mathcal{O}(n)$。


In [ ]:
class Solution:
    def kthSmallestPrimeFraction(self, A, K: int):
        n = len(A)
        left, right = 0, 1.0
        while left < right:
            mid = (left + right) / 2
            count = 0
            j = 0
            mx = [0, 1]
            for i in range(n):
                while j<n and A[i]/A[j]>mid: 
                    j+=1
                count += n - j
                if j<n and A[i]/A[j]>mx[0]/mx[1]:
                    mx = [A[i], A[j]]
            if count<K: left = mid
            elif count ==K: return mx
            else: right = mid

In [ ]:
class Solution {
    public int[] kthSmallestPrimeFraction(int[] A, int K) {
        int n = A.length;
        double left = 0, right = 1, mid;
        while (true) {
            mid = (left + right) / 2;
            int count = 0;
            int j = 0;
            int[] maxIndex = {0, 1};
            for (int i = 0; i < n; i++) {
                while (j < n && A[i] >= A[j] * mid) ++j;
                count += n - j;
                if (j < n && A[i] * maxIndex[1] > A[j] * maxIndex[0]) {
                    maxIndex = new int[]{A[i], A[j]};
                }
            }
            if (count > K) right = mid;
            else if (count < K) left = mid;
            else return maxIndex;
        }
    }
}